In [ ]:
allSamples_geno = open(snakemake.input["as_geno"], "r").readlines()

In [ ]:
split_index_dict = {n: [] for n in snakemake.params["splits"]}
num_splits = len(snakemake.params["splits"])
idx_array = [x for x in range(len(allSamples_geno))]

for n in range(num_splits):
    idx = [x for x in range(n, len(idx_array), num_splits)]
    split_index_dict[n] = [idx_array[i] for i in idx]

In [ ]:
out_prefix = snakemake.params["out_prefix"]
splits_out = f"{out_prefix}/allSamples/splits"
if not os.path.exists(splits_out):
    os.makedirs(splits_out)
for split, geno in split_index_dict.items():
    as_geno_out = f"{splits_out}/allSamples_{split}.geno"
    with open(as_geno_out, "w") as fout:
        for g in geno:
            fout.write(allSamples_geno[g])

In [ ]:
order_out = f"{out_prefix}/allSamples/site_order"
if not os.path.exists(order_out):
    os.makedirs(order_out)
site_order = open(snakemake.input["site_order"], "r").readlines()
for split, geno in split_index_dict.items():
    site_order_out = f"{order_out}/site_order_{split}.txt"
    with open(site_order_out, "w") as fout:
        for g in geno:
            fout.write(site_order[g])